### Model Trainer

In [1]:
import os
%pwd

'/Users/tranminhanh/Downloads/Data Science/TextSummarization/research'

In [2]:
os.chdir("../")
%pwd

'/Users/tranminhanh/Downloads/Data Science/TextSummarization'

### Model Trainer Configuration

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass 
class ModelTrainerConfig: 
    root_dir : Path 
    data_path : Path 
    model_ckpt : Path 
    num_train_epochs : int 
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: int
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int 

### Configuration Manager

In [4]:
from src.TextSummarization.constants import *
from src.TextSummarization.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
    
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig : 
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config

### Components

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments,Trainer 
from transformers import DataCollatorForSeq2Seq 
import torch 
from datasets import load_from_disk 

/Users/tranminhanh/Downloads/Data Science/TextSummarization/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-24 17:59:10,901: INFO: config: PyTorch version 2.6.0 available.]


In [9]:
class ModelTrainer:
    def __init__(self,config : ModelTrainerConfig):
        self.config = config 

    def train(self):
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model= model_pegasus)
        
        #Load the data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        trainer.train() 

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
        

In [ ]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()